In [31]:
import torch
import torch.nn as nn   
import torch.nn.functional as  F

class Attn(nn.Module):
    def __init__(self,query_size,key_size,value_size1,value_size2,output_size):
        """初始化函数中的参数有5个, query_size代表query的最后一维大小
           key_size代表key的最后一维大小, value_size1代表value的导数第二维大小, 
           value = (1, value_size1, value_size2)
           value_size2代表value的倒数第一维大小, output_size输出的最后一维大小"""
        super(Attn,self).__init__()
        #传入参数到类内
        self.query_size=query_size
        self.key_size=key_size
        self.value_size=value_size1
        self.value_size=value_size2
        self.output_size=output_size

        #初始线性层
        self.attn = nn.Linear(self.query_size+self.key_size,value_size1)
        self.attn_combine = nn.Linear(self.query_size+value_size2,output_size)

    def forward(self,Q,K,V):
        """forward函数的输入参数有三个, 分别是Q, K, V, 根据模型训练常识, 输入给Attion机制的
        张量一般情况都是三维张量, 因此这里也假设Q, K, V都是三维张量"""
        # 第一步, 按照计算规则进行计算, 
        # 我们采用常见的第一种计算规则
        # 将Q，K进行纵轴拼接, 做一次线性变化, 最后使用softmax处理获得结果

        attn_weights=F.softmax(self.attn(torch.cat((Q[0],K[0]),1)),dim=1)
        # 然后进行第一步的后半部分, 将得到的权重矩阵与V做矩阵乘法计算, 
        # 当二者都是三维张量且第一维代表为batch条数时, 则做bmm运算
        attn_applied= torch.bmm(attn_weights.unsqueeze(0),V)
        # 之后进行第二步, 通过取[0]是用来降维, 根据第一步采用的计算方法, 
        # 需要将Q与第一步的计算结果再进行拼接
        output=torch.cat((Q[0],attn_applied[0]),1)
        # 最后是第三步, 使用线性层作用在第三步的结果上做一个线性变换并扩展维度，得到输出
        # 因为要保证输出也是三维张量, 因此使用unsqueeze(0)扩展维度
        output=self.attn_combine(output).unsqueeze(0)
        return output,attn_weights

query_size=32
key_size=32
value_size1=32
value_size2=64
output_size=64
attn=Attn(query_size=query_size,key_size=key_size,value_size1=value_size1,value_size2=value_size2,output_size=output_size)
Q=torch.randn(1,1,32)
K=torch.randn(1,1,32)
V=torch.randn(1,32,64)
output,attn_weights=attn(Q,K,V)
print(output,attn_weights)



tensor([[[ 0.1495,  0.3650, -0.0710, -0.3783, -0.5297,  0.2371, -0.7726,
           0.1727,  0.0217, -0.4622, -0.3023, -0.6713, -0.5723, -0.1507,
          -0.7863,  0.4471, -0.2495, -0.2676,  0.4894, -0.2547,  0.7494,
          -0.2545, -0.0914, -0.0052,  0.2708,  0.1040, -0.2316,  0.5949,
           0.5094,  0.2641,  0.3962, -0.3843, -0.6380, -0.0831,  1.1316,
           0.0128, -0.1911,  0.4362,  0.2413,  0.3095, -0.4566, -0.1453,
          -0.2356,  0.4359,  0.3121, -0.3590, -0.2367,  0.2886, -0.6820,
          -0.3155,  0.4305,  0.2279, -0.0580, -0.1758, -0.4496,  0.9370,
           0.1314,  0.7020,  0.3381,  0.0032, -0.3821,  0.4225, -0.2567,
           0.0933]]], grad_fn=<UnsqueezeBackward0>) tensor([[0.0193, 0.0623, 0.0542, 0.0099, 0.0456, 0.1010, 0.0522, 0.0690, 0.0273,
         0.0136, 0.0558, 0.0176, 0.0232, 0.0186, 0.0270, 0.0540, 0.0217, 0.0130,
         0.0100, 0.0195, 0.0100, 0.0137, 0.0250, 0.0344, 0.0214, 0.0210, 0.0083,
         0.0156, 0.0192, 0.0416, 0.0635, 0.0112]